In [ ]:
#İmporting necessarty libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import tensorflow as tf
from sklearn.model_selection import train_test_split
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
#Reading the Forest Fire Data

df1 = pd.read_csv('Fire_Data.csv')
df2 = pd.read_csv('Non_Fire_Data.csv')
#big_data = pd.read_csv('Data_All.csv')
df = pd.concat([df1, df2])

In [ ]:
#Renaming the columns in the data for a better understanding

df =df.rename(columns={"Power_Lines_Proximity":"Power Lines Proximity", 
                   "ESA_Worldcover":"LULC",
                   "Forest_Density":"TCD",
                   "Forest_Road_Proximity": "Forest Road Proximity",
                   "Railroad_Proximity": "Railway Proximity",
                   "Road_Proximity": "Road Proximity",
                   "Settlement_Density": "Settlement Density",
                   "Settlement_Proximity": "Settlement Proximity",
                   "Tree_Ages": "Tree Ages",
                   "Tree_Types": "Tree Types",
                   "Water_Proximity": "Water Proximity",
                   "DEM": "Elevation",
                   "Fuels" : "Forest Type"   
                  })

labels = np.asarray(df.Fires) # Read Label Columns as numpy array for Sklearn to understand
feature_cols = ["Aspect",
                'Elevation',
                "LULC",
                "TCD",
                "Forest Type",
                "MNDWI",
                "NDVI",
                "Power Lines Proximity", 
                "Road Proximity",
                "Settlement Density",
                "Settlement Proximity",
                "Tree Types",
                "Tree Ages",
                "Forest Road Proximity",
                "Slope",
                "Water Proximity"                
                                ]

In [ ]:
# Creating X and Y values for splitting data
labels = np.asarray(df.Fires) # Read Label Columns as numpy array for Sklearn to understand
x = df[feature_cols] #Reading Features from the selected columns in df
y = labels # Reading labels as y
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2) # Split the data

In [ ]:
# ANN

Y_train = y_train
Y_test = y_test

X_train = x_train.iloc[:,0:16].values
X_test  = x_test.iloc[:,0:16].values

#X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
#Performing Feature Scaling for ANN 


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=8,activation="relu"))
ann.add(tf.keras.layers.Dense(units=8,activation="relu"))
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

#Performing Accuracy measurement functions
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['acc',f1_m,precision_m, recall_m])

ann.fit(X_train,Y_train,batch_size=32,epochs = 100)



In [ ]:
#Saving the ANN locally for future need
ann.save("ANN.ALL_Features_last")

In [ ]:
#Visualizing the model
plot_model(ann, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
#Saving the visual
from ann_visualizer.visualize import ann_viz;

ann_viz(ann, title="My first neural network")

In [ ]:
# PLotting ROC Curve

from sklearn import metrics
#set up plotting area
#plt.figure(0).clf()
plt.figure(dpi=150)


y_pred = probas = ann.predict(X_test)
fpr, tpr, _ = metrics.roc_curve(Y_test, y_pred)
auc = round(metrics.roc_auc_score(Y_test, y_pred), 3)
plt.plot(fpr,tpr,lw=1, color='red',label=" ANN_AUC = "+str(auc))

#add legend marker='o', linestyle='dashed'

plt.title('ANN ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.xlim([-0.05, 0.51])
plt.ylim([-0.05, 1.05])
plt.legend(loc='best')
plt.savefig('ROC Curve',dpi=300);

In [ ]:
#Reading the data of whole study area to implement the model and predict forest fire susceptibility values 
big_data = pd.read_csv('Data_All.csv')

In [ ]:
#Renaming the columns in the big data as before 
big_data = big_data.rename(columns={"Power_Lines_Proximity":"Power Lines Proximity", 
                   "ESA_Worldcover":"LULC",
                   "Forest_Density":"TCD",
                   "Forest_Road_Proximity": "Forest Road Proximity",
                   "Railroad_Proximity": "Railway Proximity",
                   "Road_Proximity": "Road Proximity",
                   "Settlement_Density": "Settlement Density",
                   "Settlement_Proximity": "Settlement Proximity",
                   "Tree_Ages": "Tree Ages",
                   "Tree_Types": "Tree Types",
                   "Water_Proximity": "Water Proximity",
                   "DEM": "Elevation" 
                  })
big_data2 = big_data[feature_cols]

In [ ]:
#Perform scaling once again on the big data
big_data2 = big_data.iloc[:,0:17].values
big_data2  = sc.transform(big_data2)

In [ ]:
class_code = clf.predict(big_data2)            # Predictions

In [ ]:
class_code_prob = clf.predict_proba(big_data2) # Probabilities

In [ ]:
#Creating the df for for the prediction values
Results = pd.DataFrame(class_code_prob, columns = ['Class_0_Prob','Class_1_Prob']) # Writing Probability results into specified columns

In [ ]:
#Importing predicted values into df
Results2 = pd.DataFrame(class_code_ann, columns = ['PredictionsANN']) # Writing Probability results into specified columns
Results2['Predictions_ANN'] = class_code_ann.tolist() # Writing Predictions into Column

In [ ]:
#Importing the X and Y coordinates to map the predicticted values 
Results2['X'] = big_data['X'].tolist()        # Writing X Coordinates into column
Results2['Y'] = big_data['Y'].tolist()        # Writing Y Coordinates into column

In [ ]:
#Saving the df as CSV file to map in ArcGIS
Results2.to_csv('ANN_Final.csv')           # Writing the df into CSV Table